In [147]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
import transformers
# import impact_ner_tokens as get_tokens
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np

import torch
print(torch.cuda.is_available())

label_list = ['O','B-IMPACT','I-IMPACT','B-AFFECTED','I-AFFECTED','B-SEVERITY','I-SEVERITY','B-LOCATION','I-LOCATION','B-MODIFIER','I-MODIFIER']

tokenizer = AutoTokenizer.from_pretrained('./impact-ner.model/')

False


In [148]:
tokenizer

PreTrainedTokenizerFast(name_or_path='./impact-ner.model/', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [149]:

tokens = tokenizer('''Chile's Coast Struck by Strong Aftershock a Day After #Earthquake http://t.co/aEOilTzP9oRT @Ferris_McFly: @eliroth 300 prisoners reportedly escaped during #earthquake in Iquique - #Chile. Thats fucking insane!RT @ShakingEarth: Earthquake ! mb 4.1 OFFSHORE TARAPACA, CHILE http://t.co/4t2tmkY7XKRT @seismo_steve: Seismic wave recordings of last night’s M8.2 Chile #earthquake from the @livuni @LivUniEarthSci seismometer http://t.co/j…Earthquake in Chile, rumbling volcano in Peru. What is happening in South America? #PrayForChileChile earthquake: How loss of life, damage was minimized http://t.co/dP1wIe4oGGRT @iankeithtom: I just heard. I have friends in Chile. Tell me you're OK #earthquakeMT @HumanityRoad's situation report on the earthquake in #Chile. http://t.co/PiYIBkDNUK |#terremotoMy NBC News report from Northern Chile - Earthquake Damages Chilean Infrastructure  http://t.co/Fd0i5o0G2gM5.0 OFFSHORE TARAPACA, CHILE Depth 25.3km Apr 05, 2014 00:33:57 UTC, Apr 04, 2014 20:33:57 at epicenter http://t.co/sbXn47tzj8RT @NewEarthquake: 4.8 earthquake, 96km WNW of Iquique, Chile. Apr 3 03:28 at epicenter (37m ago, depth 15km). http://t.co/dJgNhlLOVLMassive Chile Earthquake May Not Be the "Big One" http://t.co/Y7oFOAhcLyBREAKING: A 7.8-magnitude earthquake struck off Chile's northern coast Wednesday night, triggering a tsunami... http://t.co/WRbyO79to2M5.5  - 66km SSW of Iquique, Chile 2014-04-03 05:51:44 UTC (via @usgs) http://t.co/9l4tmSOX0uChile awakens to see 90''')
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

Token indices sequence length is longer than the specified maximum sequence length for this model (1074 > 512). Running this sequence through the model will result in indexing errors


torch.Size([1, 1074])

In [150]:
model = AutoModelForTokenClassification.from_pretrained('./impact-ner.model/', num_labels=len(label_list))

In [151]:
import torch
preds = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))

RuntimeError: The size of tensor a (1074) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
preds = torch.argmax(preds.logits.squeeze(), axis=1)

In [ ]:
sentence = '''Chile's Coast Struck by Strong Aftershock a Day After #Earthquake http://t.co/aEOilTzP9oRT @Ferris_McFly: @eliroth 300 prisoners reportedly escaped during #earthquake in Iquique - #Chile. Thats fucking insane!RT @ShakingEarth: Earthquake ! mb 4.1 OFFSHORE TARAPACA, CHILE http://t.co/4t2tmkY7XKRT @seismo_steve: Seismic wave recordings of last night’s M8.2 Chile #earthquake from the @livuni @LivUniEarthSci seismometer http://t.co/j…Earthquake in Chile, rumbling volcano in Peru. What is happening in South America? #PrayForChileChile earthquake: How loss of life, damage was minimized http://t.co/dP1wIe4oGGRT @iankeithtom: I just heard. I have friends in Chile. Tell me you're OK #earthquakeMT @HumanityRoad's situation report on the earthquake in #Chile. http://t.co/PiYIBkDNUK |#terremotoMy NBC News report from Northern Chile - Earthquake Damages Chilean Infrastructure  http://t.co/Fd0i5o0G2gM5.0 OFFSHORE TARAPACA, CHILE Depth 25.3km Apr 05, 2014 00:33:57 UTC, Apr 04, 2014 20:33:57 at epicenter http://t.co/sbXn47tzj8RT @NewEarthquake: 4.8 earthquake, 96km WNW of Iquique, Chile. Apr 3 03:28 at epicenter (37m ago, depth 15km). http://t.co/dJgNhlLOVLMassive Chile Earthquake May Not Be the "Big One" http://t.co/Y7oFOAhcLyBREAKING: A 7.8-magnitude earthquake struck off Chile's northern coast Wednesday night, triggering a tsunami... http://t.co/WRbyO79to2M5.5  - 66km SSW of Iquique, Chile 2014-04-03 05:51:44 UTC (via @usgs) http://t.co/9l4tmSOX0uChile awakens to see 90'''

In [ ]:
len(sentence.split(' '))

26

In [ ]:
preds.size()

torch.Size([62])

In [ ]:
len(tokens['input_ids'])

62

In [ ]:
words = tokenizer.batch_decode(tokens['input_ids'])

In [ ]:
value_preds = [label_list[i] for i in preds]

In [ ]:
import pandas as pd
pd.DataFrame({'ner': value_preds, 'words': words}).to_csv('impact_ner.csv')